# Tohoku tsunami inversion

Try solving the shallow water equations *without* using a mixed space.
Instead, use a vector.

In [1]:
%matplotlib inline

In [2]:
from thetis import *
from firedrake_adjoint import *

In [3]:
import matplotlib.pyplot as plt

In [4]:
from adapt_utils.case_studies.tohoku.options.options import TohokuInversionOptions
from adapt_utils.misc import gaussian, ellipse

In [5]:
def plot_field(field, figure=None, axes=None, height=5, **kwargs):
    CG1 = field.ufl_element().degree() == 1 and field.ufl_element().family() == 'Lagrange'
    fs_constructor = FunctionSpace if len(field.function_space().shape) == 0 else VectorFunctionSpace
    out = field if CG1 else interpolate(field, fs_constructor(field.function_space().mesh(), "CG", 1))
    
    if figure is None and axes is None:
        figure, axes = plt.subplots(figsize=(height*1.2, height))
    if axes is None:
        axes = plt.gca()
    figure.colorbar(tricontourf(out, levels=50, axes=axes, cmap='coolwarm', **kwargs), ax=axes);
    axes.axis(False);
    axes.set_title(field.dat.name);
    return axes

### Forward model

In [6]:
level = 0
op = TohokuInversionOptions(level=level)
gauges = list(op.gauges.keys())
for gauge in gauges:
    if gauge[:2] not in ('P0','80'):
        op.gauges.pop(gauge)
gauges = list(op.gauges.keys())
print(gauges)
op.end_time = 60*30

['801', '802', '803', '804', '806', '807', 'P02', 'P06']


In [7]:
mesh = op.default_mesh
P1DG = FunctionSpace(mesh, VectorElement("DG", triangle, 1, 3))
P1 = FunctionSpace(mesh, "CG", 1)
P0 = FunctionSpace(mesh, "DG", 0)

In [8]:
b = Function(P1).assign(op.set_bathymetry(P1))
g = Constant(op.g)
f = Function(P1).assign(op.set_coriolis(P1))

In [9]:
dtc = Constant(op.dt)
n = FacetNormal(mesh)

q = TrialFunction(P1DG)
u, v, eta = q
xi = TestFunction(P1DG)
phi, psi, zeta = xi

q_ = Function(P1DG)
u_, v_, eta_ = q_

a = inner(q, xi)*dx
L = inner(q_, xi)*dx


def G(uu, vv, elev):
    """
    **HARD-CODED** formulation for vector equal-order LSWE.
    
    Uses the same flux terms as Thetis.
    """
    uv = as_vector([uu, vv])
    uv_test = as_vector([phi, psi])
    hear_star = avg(elev) + sqrt(b/g)*jump(uv, n)
    hu_star = b*(avg(uv) + sqrt(g/b)*jump(elev, n))
    c = sqrt(g*b)
    
    # Gravity
    F = -g*elev*nabla_div(uv_test)*dx
    F += g*hear_star*jump(as_vector([phi, psi]), n)*dS
    F += c*dot(uv, n)*dot(uv_test, n)*ds
    F += 0.5*g*elev*dot(uv_test, n)*ds(100)
    
    # Coriolis
    F += -f*inner(phi, vv)*dx
    F += f*inner(psi, uu)*dx
    
    # HUDiv
    F += -inner(grad(zeta), b*uv)*dx
    F += inner(jump(zeta, n), b*hu_star)*dS
    F += 0.5*zeta*b*dot(uv, n)*ds
    F += zeta*c*elev*ds(100)
    
    return F


a += 0.5*dtc*G(u, v, eta)
L += -0.5*dtc*G(u_, v_, eta_)

q = Function(P1DG)
u, v, eta = q

params = {
    "mat_type": "aij",
    "snes_type": "ksponly",
    "ksp_type": "preonly",
    "pc_type": "lu",
    "pc_factor_mat_solver_type": "mumps",
}
prob = LinearVariationalProblem(a, L, q)
solver = LinearVariationalSolver(prob, solver_parameters=params)

# Source model

In [10]:
q0 = Function(P1DG)
loc = (0.7e+06, 4.2e+06)
radii = (48e+03, 96e+03)
angle = pi/12
q0.interpolate(as_vector([0, 0, 10*gaussian([loc + radii], mesh, rotation=angle)]))

Coefficient(WithGeometry(FunctionSpace(<firedrake.mesh.MeshTopology object at 0x7fba3ffd6978>, VectorElement(FiniteElement('Discontinuous Lagrange', triangle, 1), dim=3), name=None), Mesh(VectorElement(FiniteElement('Lagrange', triangle, 1), dim=2), 1)), 128)

## Forward solve

Now run the forward model, time integrating the QoI as time progresses.

In [11]:
def solve_forward(init):
    q_.assign(init)
    t = 0.0
    J = 0
    iteration = 0
    wq = Constant(0.5)
    eta_obs = Constant(0.0)
    for gauge in gauges:
        op.gauges[gauge]['init'] = None
        if t < op.gauges[gauge]['arrival_time']:
                continue
        op.gauges[gauge]['init'] = q_.at(op.gauges[gauge]['coords'])[2]
        eta_obs.assign(op.gauges[gauge]['init'])
        J = J + assemble(wq*op.gauges[gauge]['indicator']*(q_[2] - eta_obs)**2*dx)
    while t < op.end_time:
        print("t = {:2.0f} mins".format(t/60))

        # Solve forward equation at current timestep
        solver.solve()
        q_.assign(q)
        t += op.dt
        iteration += 1

        # Time integrate QoI
        for gauge in op.gauges:
            if t < op.gauges[gauge]['arrival_time']:
                continue
            elif np.isclose(t, op.gauges[gauge]['arrival_time']):
                wq.assign(0.5*0.5*op.dt)
            elif np.isclose(t, op.gauges[gauge]['departure_time']):
                wq.assign(0.5*0.5*op.dt)
            elif t > op.gauges[gauge]['departure_time']:
                continue
            else:
                wq.assign(0.5*1.0*op.dt)
                
            # Interpolate observations
            if op.gauges[gauge]['init'] is None:
                op.gauges[gauge]['init'] = q.at(op.gauges[gauge]['coords'])[2]
            eta_obs.assign(float(op.gauges[gauge]['interpolator'](t)) + op.gauges[gauge]['init'])

            # Continuous form of error
            J = J + assemble(wq*op.gauges[gauge]['indicator']*(q[2] - eta_obs)**2*dx)

    assert np.allclose(t, op.end_time), "mismatching end time ({:.2f} vs {:.2f})".format(t, op.end_time)
    return J

Define gauge indicators

In [12]:
radius = 20.0e+03*pow(0.5, level)  # The finer the mesh, the more precise the indicator region
for gauge in gauges:
    loc = op.gauges[gauge]["coords"]
    op.gauges[gauge]['indicator'] = interpolate(ellipse([loc + (radius,)], mesh), P0)
    area = assemble(op.gauges[gauge]['indicator']*dx)
    op.gauges[gauge]['indicator'].assign(op.gauges[gauge]['indicator']/area)
    op.sample_timeseries(gauge, sample=op.gauges[gauge]['sample'], detide=True)

In [13]:
J = solve_forward(q0)
print("Quantity of interest = {:.4e}".format(J))

t =  0 mins


Error: error code 77
[0] SNESSolve() line 4515 in /home/joe/software/firedrake-no-adapt/src/petsc/src/snes/interface/snes.c
[0] SNESSolve_KSPONLY() line 51 in /home/joe/software/firedrake-no-adapt/src/petsc/src/snes/impls/ksponly/ksponly.c
[0] KSPSolve() line 882 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/ksp/interface/itfunc.c
[0] KSPSolve_Private() line 662 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/ksp/interface/itfunc.c
[0] KSPSetUp() line 405 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/ksp/interface/itfunc.c
[0] PCSetUp() line 959 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/pc/interface/precon.c
[0] PCSetUp_FieldSplit() line 599 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/pc/impls/fieldsplit/fieldsplit.c
[0] PCFieldSplitSetDefaults() line 556 in /home/joe/software/firedrake-no-adapt/src/petsc/src/ksp/pc/impls/fieldsplit/fieldsplit.c
[0] Petsc has generated inconsistent data
[0] Unhandled case, must have a

In [ ]:
eta = interpolate(q[2], P1)
eta.dat.name = 'Free surface'
plot_field(eta)

## Discrete adjoint

Note that we only Taylor test the elevation and effectively zero out the perturbation away from the source.

In [ ]:
c = Control(q0)

In [ ]:
stop_annotating();

In [ ]:
Jhat = ReducedFunctional(J, c)

In [ ]:
np.random.seed(0)
m0 = Function(q0)
dm0 = Function(P1DG)
dm0.dat.data[:] = np.random.rand(*dm0.dat.data.shape)*m0.dat.data
m0 += dm0
minconv = taylor_test(Jhat, m0, dm0)
assert minconv > 1.90, minconv